In [1]:
#导入所有的依赖包
import  tensorflow as tf
import numpy as np
from cnnModel import cnnModel
import os
import pickle
import getConfig
import sys
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [2]:
#初始化一个字典，用于存储从配置文件中读取的参数配置
gConfig = {}
#使用get_config方法获取配置文件中的参数
gConfig=getConfig.get_config(config_file="config.ini")

In [3]:
#定义read_data函数，读取数据
def read_data(dataset_path, im_dim_x, im_dim_y, num_channels,num_files,images_per_file):
         # 获取训练集中训练文件的名称
        files_names = os.listdir(dataset_path)
        print(files_names)
        #创建空的多维数组用于存放图片的二进制数据
        dataset_array = np.zeros(shape=(num_files * images_per_file, im_dim_y, im_dim_x, num_channels))
        # 创建空的数组用于存放图片的标注信息
        dataset_labels = np.zeros(shape=(num_files * images_per_file), dtype=np.uint8)
        index = 0
        #从训练集中读取二进制数据并将其维度转换成300*480*3
        for file_name in files_names:
            if file_name[0:11] == "data_batch_":
                print("正在处理数据 : ", file_name)
                data_dict = unpickle_patch(dataset_path + file_name)
                images_data = data_dict["data"]
                print(images_data.shape)
                # 格式转换为300*480*3 shape.
                images_data_reshaped = np.reshape(images_data, newshape=(len(images_data), im_dim_y, im_dim_x, num_channels))
                # 将维度转换后的图片数据存入指定数组内
                dataset_array[index * images_per_file:(index + 1) * images_per_file, :, :, :] = images_data_reshaped
                #  将维度转换后的标注数据存入指定数组内
                dataset_labels[index * images_per_file:(index + 1) * images_per_file] = data_dict["labels"]
                index = index + 1
        return dataset_array, dataset_labels  # 返回数据

In [4]:
def unpickle_patch(file):
    #打开文件，读取二进制文件，返回读取到的数据
    patch_bin_file = open(file, 'rb')
    patch_dict = pickle.load(patch_bin_file, encoding='bytes')
    return patch_dict

In [5]:
def create_model():
    #判断是否有预训练模型
    if 'pretrained_model'in gConfig:
        model=tf.keras.models.load_model(gConfig['pretrained_model'])
        return model
    ckpt=tf.io.gfile.listdir(gConfig['working_directory'])

    #判断是否已经有model文件存在，如果model文件存在则加载原来的model并在原来的moldel继续训练，如果不存在则新建model相关文件
    if ckpt:
        #如果存在模型文件，则加载存放model文件夹中最新的文件
        model_file=os.path.join(gConfig['working_directory'], ckpt[-1])
        print("Reading model parameters from %s" % model_file)
        #使用tf.keras.models.load_model来加载模型

        model=tf.keras.models.load_model(model_file)
        return model
    else:
        model=cnnModel(gConfig['keeps'])
        model=model.createModel()
        return model

In [7]:
#使用read_data函数读取训练数据
dataset_array, dataset_labels = read_data(dataset_path=gConfig['dataset_path'], im_dim_x=gConfig['im_dim_x'], im_dim_y=gConfig['im_dim_y'],
   num_channels=gConfig['num_channels'],num_files=gConfig['num_files'],images_per_file=648)

['data_batch_0_480_300', 'data_batch_0_480_440', 'data_batch_0_480_320', 'data_batch_0_480_640']
正在处理数据 :  data_batch_0_480_300
(648, 432000)
正在处理数据 :  data_batch_0_480_440
(648, 432000)
正在处理数据 :  data_batch_0_480_320
(648, 432000)
正在处理数据 :  data_batch_0_480_640
(648, 432000)


In [8]:
#对读取到的输入数据进行归一化处理
dataset_array = dataset_array.astype('float32')
dataset_array = dataset_array / 255

In [10]:
#使用read_data函数读取训练数据
test_array, test_labels = read_data(dataset_path=gConfig['test_path'], im_dim_x=gConfig['im_dim_x'], im_dim_y=gConfig['im_dim_y'],
   num_channels=gConfig['num_channels'],num_files=4,images_per_file=162)

['data_batch_0_480_300', 'data_batch_0_480_440', 'data_batch_0_480_320', 'data_batch_0_480_640']
正在处理数据 :  data_batch_0_480_300
(162, 432000)
正在处理数据 :  data_batch_0_480_440
(162, 432000)
正在处理数据 :  data_batch_0_480_320
(162, 432000)
正在处理数据 :  data_batch_0_480_640
(162, 432000)


In [11]:
test_array = test_array.astype('float32') / 255

In [30]:
all_histories = []
if os.path.exists('./boring_out.csv'):
    with open('./boring_out.csv', encoding='utf-8') as f:
        csv_reader_lines = csv.reader(f)
        for one_line in csv_reader_lines:
            all_histories.append(one_line)
else:
    all_histories = [['epoch', 'loss', 'mae', 'val_lose', 'val_mae']]
print(all_histories[-1])

['101', '8.244348558', '2.3012226', '167.0393372', '10.312181']


In [32]:
#定义训练函数
def train():
    #初始化Model
    model=create_model()
#     print(model.summary())
    #使用fit方法进行训练，verbose是控制输出的信息，validation_data是配置测试数据集
    history = model.fit(dataset_array,dataset_labels,verbose=1,epochs=1,validation_data=(test_array,test_labels),batch_size=32)
#     print(history.history)
    # 获取到history的属性
    epoch = history.epoch
    loss = history.history['loss']
    mae = history.history['mae']
    val_lose = history.history['val_loss']
    val_mae = history.history['val_mae']
    length = len(all_histories)
    for i in epoch:
        all_histories.append([int(length+i), loss[i], mae[i], val_lose[i], val_mae[i]])
    # 达到一个训练模型保存点后，将模型保存下来，并打印出这个保存点的平均准确率
    filename='cnn_model.h5'
    checkpoint_path = os.path.join(gConfig['working_directory'], filename)
    model.save(checkpoint_path)
    sys.stdout.flush()

In [33]:
train()
# 输出成csv文件
with open('./boring_out.csv', mode='w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(all_histories)
print(all_histories[-1])

Reading model parameters from model_dir/boring/cnn_model.h5
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 300, 480, 64)      1792      
_________________________________________________________________
conv2 (Conv2D)               (None, 300, 480, 64)      36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 300, 480, 64)      256       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 150, 240, 64)      0         
_________________________________________________________________
d1 (Dropout)                 (None, 150, 240, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 240, 64)      256       
______________________________________________________________